# BM-25

In [15]:
! pip install rank-bm25

In [16]:
import pandas as pd
from rank_bm25 import BM25Okapi

In [21]:
base_url = "https://raw.githubusercontent.com/p0velentius/rug-pull/main/"
websites = pd.read_csv(base_url + "preprocessing/websites_lemmatizated.csv")
questions = pd.read_csv(base_url + "preprocessing/questions_lemmatizated.csv")

In [48]:
n = 50

def tokens_from_lemmas(text):
    if pd.isna(text):
        return []
    return str(text).split()

tokenized_docs = [tokens_from_lemmas(t) for t in websites["text_lemmas"]]
bm25 = BM25Okapi(tokenized_docs)

results = []
for _, qrow in questions.iterrows():
    q_id = qrow["q_id"]
    q_lemmas = qrow["query_clean"]

    query_tokens = tokens_from_lemmas(q_lemmas)
    if not query_tokens:
        continue

    scores = bm25.get_scores(query_tokens)
    top_idx = np.argsort(scores)[::-1][:n]

    for rank, doc_idx in enumerate(top_idx, start=1):
        results.append({
            "question_id": q_id,
            "web_id": websites.loc[doc_idx, "web_id"],
            "rank_bm25": rank,
            "score_bm25": float(scores[doc_idx]),
        })

bm_25 = pd.DataFrame(results)
bm_25.to_csv("bm25_results.csv", index=False)


# BGE m3

In [ ]:
# =======================
# Block 1: Token-based chunking for BGE-m3
# =======================
# Входные объекты уже существуют:
import os
import hashlib
import pandas as pd
from tqdm.auto import tqdm
from transformers import AutoTokenizer
base_url = "https://raw.githubusercontent.com/p0velentius/rug-pull/main/"
web       = pd.read_csv(base_url + "preprocessing/websites_preprocessed.csv")
questions = pd.read_csv(base_url + "preprocessing/questions_preprocessed.csv")
sample    = pd.read_csv(base_url + "sample_submission.csv")

!pip -q install transformers>=4.44.0 sentencepiece tqdm pyarrow



# ----------- Config (можно править) -----------
MODEL_NAME = "BAAI/bge-m3"        # токенизатор как у будущей модели эмбеддингов
CHUNK_TOKENS = 256                # целевой размер чанка в токенах (по тексту, без заголовка)
CHUNK_OVERLAP = 50                # перекрытие в токенах
MAX_SEQ_LEN = 512                 # лимит модели при эмбеддинге
INCLUDE_TITLE = True              # подставлять заголовок в каждый чанк
USE_PREFIX = True                 # добавлять префикс "passage: "
MAX_CHARS_TEXT = 4000             # мягкая отсечка по символам перед токенизацией
ARTIFACT_DIR = "./artifacts"
CHUNKS_PARQUET = os.path.join(ARTIFACT_DIR, "doc_chunks.parquet")
os.makedirs(ARTIFACT_DIR, exist_ok=True)

# ----------- Колонки в web (автоопределение) -----------
title_col = "title_clean" if "title_clean" in web.columns else ("title" if "title" in web.columns else None)
text_col  = "text_clean"  if "text_clean"  in web.columns else ("text"  if "text"  in web.columns else None)
id_col    = "web_id"      if "web_id"      in web.columns else ("id"    if "id"    in web.columns else None)

assert id_col is not None,    "Не найден идентификатор (ожидаю колонку 'web_id' или 'id')"
assert title_col is not None, "Не найден заголовок (ожидаю 'title_clean' или 'title')"
assert text_col  is not None, "Не найден текст (ожидаю 'text_clean' или 'text')"

# ----------- Tokenizer -----------
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def md5(s: str) -> str:
    return hashlib.md5(s.encode("utf-8")).hexdigest()

def split_into_token_chunks(text: str, chunk_tokens: int, overlap: int):
    """
    Разбиваем ТОЛЬКО text (без title/prefix) на токен-чанки.
    Возвращаем список словарей: {start_idx, end_idx, chunk_text, n_tokens}
    """
    text = (text or "").strip()
    if MAX_CHARS_TEXT and len(text) > MAX_CHARS_TEXT:
        text = text[:MAX_CHARS_TEXT]

    # Токенизируем без спец-токенов
    ids = tok.encode(text, add_special_tokens=False)
    if not ids:
        return []

    # Защита от некорректных параметров
    chunk_tokens = max(32, int(chunk_tokens))
    overlap = max(0, min(int(overlap), chunk_tokens - 1))

    stride = chunk_tokens - overlap
    chunks = []
    n = len(ids)
    start = 0
    while start < n:
        end = min(start + chunk_tokens, n)
        sub_ids = ids[start:end]
        chunk_text = tok.decode(sub_ids, skip_special_tokens=True)
        chunks.append({
            "start_tok": start,
            "end_tok": end,
            "text_chunk": chunk_text,
            "text_tokens": len(sub_ids),
        })
        if end >= n:
            break
        start += stride
    return chunks

def build_passage(title: str, chunk_text: str, include_title=True, use_prefix=True):
    """
    Собираем итоговую строку чанка, которая пойдёт в эмбеддинг:
    (Важно: title НЕ участвует в разбиении на чанки, но добавляется в каждый чанк)
    """
    title = (title or "").strip()
    base = f"{title}\n\n{chunk_text}" if (include_title and title) else chunk_text
    out = f"passage: {base}" if use_prefix else base
    return out

def ensure_seq_len_budget(passage: str, budget: int = MAX_SEQ_LEN) -> str:
    """
    Страховка: если в редком случае passage > MAX_SEQ_LEN по токенам,
    обрежем хвост chunk_text, сохранив title и префикс.
    """
    ids = tok.encode(passage, add_special_tokens=True)
    if len(ids) <= budget:
        return passage

    # Грубая обрезка только хвоста (title остаётся)
    # Разделим по двойному переносу (title \n\n chunk)
    if "\n\n" in passage:
        title_part, chunk_part = passage.split("\n\n", 1)
    else:
        title_part, chunk_part = "", passage

    # Сколько токенов «съедают» заголовок и префикс
    prefix = "passage: " if USE_PREFIX else ""
    head = f"{prefix}{title_part}\n\n" if title_part else prefix
    head_ids = tok.encode(head, add_special_tokens=False)
    budget_for_chunk = max(32, budget - len(head_ids) - 4)

    chunk_ids = tok.encode(chunk_part, add_special_tokens=False)
    chunk_ids = chunk_ids[:budget_for_chunk]
    safe_chunk = tok.decode(chunk_ids, skip_special_tokens=True)
    return f"{head}{safe_chunk}"

def make_doc_chunks_df(web_df: pd.DataFrame) -> pd.DataFrame:
    """
    Главная функция блока 1:
    - чанкование по токенам text_clean
    - добавление title в каждый чанк
    - контроль MAX_SEQ_LEN для будущего эмбеддинга
    - сборка датафрейма с метаданными
    """
    rows = []
    for _, row in tqdm(web_df.iterrows(), total=len(web_df), desc="Chunking"):
        web_id = str(row[id_col])
        title  = str(row[title_col]) if pd.notna(row[title_col]) else ""
        text   = str(row[text_col])  if pd.notna(row[text_col])  else ""

        chunks = split_into_token_chunks(text, CHUNK_TOKENS, CHUNK_OVERLAP)
        if not chunks:
            # Создадим пустышку-чанк (на случай совсем пустого текста)
            chunks = [{"start_tok": 0, "end_tok": 0, "text_chunk": "", "text_tokens": 0}]

        for j, ch in enumerate(chunks):
            passage = build_passage(title, ch["text_chunk"], INCLUDE_TITLE, USE_PREFIX)
            passage = ensure_seq_len_budget(passage, MAX_SEQ_LEN)

            rows.append({
                "web_id": web_id,
                "chunk_id": j,
                "start_tok": ch["start_tok"],
                "end_tok": ch["end_tok"],
                "text_tokens": ch["text_tokens"],
                "title": title,
                "text_chunk": ch["text_chunk"],
                "passage": passage,
                "passage_tok_len": len(tok.encode(passage, add_special_tokens=True)),
                "passage_hash": md5(passage),
            })

    df_chunks = pd.DataFrame(rows)
    # полезные индексы/сортировк
    return df_chunks

# ----------- Run Block 1 -----------
doc_chunks = make_doc_chunks_df(web)
doc_chunks.to_parquet(CHUNKS_PARQUET, index=False)
print(f"✅ Chunks saved -> {CHUNKS_PARQUET}")
print(doc_chunks.head(3))
print(f"Docs: {web.shape[0]}  |  Chunks total: {doc_chunks.shape[0]}  |  Avg chunks/doc: {doc_chunks.shape[0] / max(1, web.shape[0]):.2f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Chunking:   0%|          | 0/1938 [00:00<?, ?it/s]

✅ Chunks saved -> ./artifacts/doc_chunks.parquet
  web_id  chunk_id  start_tok  end_tok  text_tokens  \
0      1         0          0       98           98   
1      2         0          0      256          256   
2      2         1        206      462          256   

                                               title  \
0  альфа банк кредитные дебетовые карты кредиты н...   
1                         клуб деньги имеют значение   
2                         клуб деньги имеют значение   

                                          text_chunk  \
0  рассчитайте выгоду расчёт калькулятора предвар...   
1  брокерские услуги открытие брокерского счёта п...   
2  карты привилегиями кэшбэком до 15 драгоценные ...   

                                             passage  passage_tok_len  \
0  passage: альфа банк кредитные дебетовые карты ...              134   
1  passage: клуб деньги имеют значение\n\nброкерс...              264   
2  passage: клуб деньги имеют значение\n\nкарты п...         

In [ ]:
# ============================================
# Block 2: Embeddings для чанков + поиск/дедуп
# ============================================
!pip -q install "sentence-transformers>=2.6.1" faiss-cpu tqdm pyarrow

import os, time
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
import torch, faiss

# ---------- Конфиг ----------
MODEL_NAME = "BAAI/bge-m3"        # single-vector через sentence-transformers
ARTIFACT_DIR = "./artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

CHUNKS_PARQUET = os.path.join(ARTIFACT_DIR, "doc_chunks.parquet")   # из блока 1
EMB_CHUNKS_NPZ = os.path.join(ARTIFACT_DIR, "chunk_embeddings_bge_m3.npz")
CHUNK_META_PARQUET = os.path.join(ARTIFACT_DIR, "chunk_meta.parquet")
FAISS_PATH = os.path.join(ARTIFACT_DIR, "faiss_bge_m3_chunks.index")

QUERY_TOPK_CHUNKS = 200     # сколько чанков достаём для каждого запроса
TOP_UNIQUE_WEB = 50         # сколько уникальных web_id вернуть (после дедупа)
USE_PREFIXES = True         # query:/passage:
MAX_SEQ_LEN = 512

# ---------- Девайс ----------
DEVICE = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
BATCH = 128 if DEVICE == "cuda" else 32

# ---------- Модель ----------
model = SentenceTransformer(MODEL_NAME, device=DEVICE)
model.max_seq_length = MAX_SEQ_LEN

# ---------- 1) Читаем чанки (из памяти или с диска) ----------
if "doc_chunks" not in globals():
    assert os.path.exists(CHUNKS_PARQUET), "Нет doc_chunks в памяти и нет файла doc_chunks.parquet"
    doc_chunks = pd.read_parquet(CHUNKS_PARQUET)

# Проверим необходимые колонки
need_cols = {"web_id", "chunk_id", "passage"}
missing = need_cols - set(doc_chunks.columns)
assert not missing, f"В doc_chunks нет колонок: {missing}"

# ---------- 2) Эмбеддинги для чанков ----------
def embed_passages(passages: list[str]) -> np.ndarray:
    emb = model.encode(
        passages,
        batch_size=BATCH,
        convert_to_numpy=True,
        normalize_embeddings=True,   # L2-норма -> можно IP как cosine
        show_progress_bar=True,
        device=DEVICE
    )
    return emb.astype("float32")

if not os.path.exists(EMB_CHUNKS_NPZ):
    t0 = time.time()
    chunk_vecs = embed_passages(doc_chunks["passage"].tolist())
    np.savez_compressed(EMB_CHUNKS_NPZ, vectors=chunk_vecs)
    # сохраним метаданные чанков отдельно (для лёгкой подгрузки)
    doc_chunks[["web_id","chunk_id"]].to_parquet(CHUNK_META_PARQUET, index=False)
    print(f"✅ Чанки закодированы: shape={chunk_vecs.shape} | saved -> {EMB_CHUNKS_NPZ} ({time.time()-t0:.1f}s)")
else:
    chunk_vecs = np.load(EMB_CHUNKS_NPZ)["vectors"].astype("float32")
    print(f"ℹ️ Загружены эмбеддинги чанков: {chunk_vecs.shape}")

# ---------- 3) FAISS-индекс (по чанкам) ----------
def build_faiss_ip(mat: np.ndarray, path: str):
    idx = faiss.IndexFlatIP(mat.shape[1])   # Inner Product = cosine (т.к. L2-норм)
    idx.add(mat)
    faiss.write_index(idx, path)
    return idx

if os.path.exists(FAISS_PATH):
    faiss_index = faiss.read_index(FAISS_PATH)
    print("ℹ️ FAISS index загружен")
else:
    faiss_index = build_faiss_ip(chunk_vecs, FAISS_PATH)
    print("✅ FAISS index построен и сохранён")

# Быстрый доступ к метаданным чанков
chunk_meta = pd.read_parquet(CHUNK_META_PARQUET) if os.path.exists(CHUNK_META_PARQUET) \
             else doc_chunks[["web_id","chunk_id"]].copy()

# Для возврата ссылок присоединим `url`, `title_clean` (если есть)
assert "web" in globals(), "Ожидаю DataFrame web в памяти"
join_cols = ["web_id", "url"]
if "title_clean" in web.columns: join_cols += ["title_clean"]
if "kind" in web.columns:        join_cols += ["kind"]
web_slim = web[join_cols].copy()
web_slim["web_id"] = web_slim["web_id"].astype(str)

# ---------- 4) Эмбеддинг запросов ----------
# Определим колонку с текстом запроса
q_text_col = "question_clean" if "question_clean" in questions.columns else \
             ("question" if "question" in questions.columns else questions.columns[1])

q_id_col = "question_id" if "question_id" in questions.columns else questions.columns[0]

def make_query_text(s: str) -> str:
    s = (s or "").strip()
    return f"query: {s}" if USE_PREFIXES else s

def embed_questions(q_df: pd.DataFrame) -> tuple[np.ndarray, pd.Series, pd.Series]:
    q_texts = [ make_query_text(x) for x in q_df[q_text_col].astype(str).tolist() ]
    q_emb = model.encode(
        q_texts,
        batch_size=BATCH,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=True,
        device=DEVICE
    ).astype("float32")
    return q_emb, q_df[q_id_col].astype(str), q_df[q_text_col].astype(str)

# ---------- 5) Поиск по чанкам ----------
def search_chunks(query_emb: np.ndarray, topk=QUERY_TOPK_CHUNKS):
    """
    query_emb: shape (d,) или (n,d)
    Возвращает:
      D: (n, topk) — score (cosine)
      I: (n, topk) — индексы чанков
    """
    if query_emb.ndim == 1:
        query_emb = query_emb[None, :]
    D, I = faiss_index.search(query_emb, topk)
    return D, I

def deduplicate_by_web(results_df: pd.DataFrame, top_unique=TOP_UNIQUE_WEB):
    """
    На входе: df с колонками ['question_id','web_id','score', ...] отсортированный по убыванию score.
    На выходе: по каждому вопросу оставляем первые вхождения уникальных web_id, максимум top_unique штук.
    """
    out = []
    for qid, group in results_df.groupby("question_id", sort=False):
        g = group.drop_duplicates("web_id", keep="first").head(top_unique)
        out.append(g)
    return pd.concat(out, ignore_index=True)

def retrieve_for_questions(q_df: pd.DataFrame,
                           topk_chunks=QUERY_TOPK_CHUNKS,
                           top_unique=TOP_UNIQUE_WEB,
                           return_for_reranker=True):
    q_emb, q_ids, q_texts = embed_questions(q_df)
    all_rows = []
    for idx in tqdm(range(q_emb.shape[0]), desc="Searching"):
        D, I = search_chunks(q_emb[idx], topk=topk_chunks)  # (1, K)
        scores = D[0]
        idxs = I[0]

        # Собираем табличку кандидатов по чанкам
        cand = pd.DataFrame({
            "chunk_row": idxs,
            "score_dense": scores,
            "rank_dense": np.arange(1, len(scores)+1, dtype=int),
        })
        # Присоединяем web_id, chunk_id
        cand = cand.merge(chunk_meta.reset_index().rename(columns={"index":"chunk_row"}),
                          on="chunk_row", how="left")

        # Присоединяем ссылку/титл
        cand = cand.merge(web_slim, on="web_id", how="left")
        cand["question_id"] = q_ids.iloc[idx]
        cand["question_text"] = q_texts.iloc[idx]

        all_rows.append(cand)

    res = pd.concat(all_rows, ignore_index=True).sort_values(
        ["question_id", "score_dense"], ascending=[True, False]
    )

    # Дедуп по web_id (оставляем лучший чанк сайта)
    res_unique = deduplicate_by_web(
        res.rename(columns={"score_dense":"score"}), top_unique=top_unique
    ).rename(columns={"score":"score_dense"})

    # Для реранкера подготовим пары (по желанию)
    rerank_df = None
    if return_for_reranker:
        # Нам нужен текст чанка. Он в doc_chunks.
        # doc_chunks может быть большим — соединим аккуратно по индексу chunk_row.
        doc_chunks_reset = doc_chunks.reset_index(drop=True)
        res_for_rerank = res_unique.merge(
            doc_chunks_reset.reset_index().rename(columns={"index":"chunk_row"})[["chunk_row","passage"]],
            on="chunk_row", how="left"
        )
        rerank_df = res_for_rerank[[
            "question_id", "question_text", "web_id", "url",
            "chunk_id", "chunk_row", "passage", "score_dense", "rank_dense"
        ]].copy()

    return res_unique, rerank_df

# ---------- 6) Запуск поиска и возврат ссылок ----------
# Пример: посчитаем кандидатов для первых N вопросов
NQ = len(questions)  # можно поставить, например, 50
res_unique, rerank_ready = retrieve_for_questions(
    questions.iloc[:NQ], topk_chunks=QUERY_TOPK_CHUNKS, top_unique=TOP_UNIQUE_WEB, return_for_reranker=True
)
CHUNK_RANKINGS_PARQUET = os.path.join(ARTIFACT_DIR, "chunk_rankings.parquet")
RERANK_CANDIDATES_PARQUET = os.path.join(ARTIFACT_DIR, "rerank_candidates.parquet")

res_unique.to_parquet(CHUNK_RANKINGS_PARQUET, index=False)
rerank_ready.to_parquet(RERANK_CANDIDATES_PARQUET, index=False)

# ============================================
# Block 2.1: Подготовка выхода BGE-m3 для RRF
# ============================================
import os
import pandas as pd
import numpy as np

# Куда сохраняем результат
DENSE_DOCS_PARQUET = os.path.join(ARTIFACT_DIR, "dense_docs.parquet")
DENSE_websites     = os.path.join(ARTIFACT_DIR, "dense_docs.csv")

# 1) Убедимся, что у нас есть res_unique и rerank_ready
#    (они возвращаются из retrieve_for_questions)
if "res_unique" not in globals() or "rerank_ready" not in globals():
    # если ещё не считали retrieve_for_questions — считаем для всех вопросов
    NQ = len(questions)  # при желании можно ограничить, например NQ = 100
    res_unique, rerank_ready = retrieve_for_questions(
        questions.iloc[:NQ],
        topk_chunks=QUERY_TOPK_CHUNKS,
        top_unique=TOP_UNIQUE_WEB,
        return_for_reranker=True
    )

# На этом этапе:
# - res_unique: уже дедупнут по web_id (оставлен лучший чанк документа)
# - rerank_ready: то же самое, но с добавленным текстом чанка в колонке 'passage'
#   колонки у rerank_ready:
#   ['question_id','question_text','web_id','url',
#    'chunk_id','chunk_row','passage','score_dense','rank_dense']

# 2) Пересчитаем ранг уже на уровне документов (web_id),
#    потому что текущий rank_dense — это ранг чанка до финальной сортировки.
dense_docs = rerank_ready.copy()
dense_docs["question_id"] = pd.to_numeric(dense_docs["question_id"], errors="raise")
dense_docs = dense_docs.sort_values(
    ["question_id", "score_dense"],
    ascending=[True, False]
).reset_index(drop=True)

# ранжируем документы (уникальные web_id) внутри каждого вопроса
dense_docs["rank_dense"] = (
    dense_docs.groupby("question_id").cumcount() + 1
)

# 3) Собираем итоговую табличку для RRF – РОВНО тот формат, который ты просил
# 3) Собираем итоговую табличку для RRF – без текстовой колонки
dense_docs = dense_docs[[
    "question_id",
    "web_id",
    "score_dense",
    "rank_dense"
]]

# 4) Сохраняем в файлы
dense_docs.to_parquet(DENSE_DOCS_PARQUET, index=False)
dense_docs.to_csv(DENSE_websites, index=False)

print(f"✅ dense_docs сохранён -> {DENSE_DOCS_PARQUET}")
print(f"✅ dense_docs сохранён -> {DENSE_websites}")
print(dense_docs.head(10))



print("✅ Готово: получили лучшие уникальные сайты на запрос (после дедупа по web_id)")
display_cols = ["question_id", "web_id", "url", "score_dense", "rank_dense"]
print(res_unique[display_cols].head(10))

print("\n(Опционально) Кандидаты для реранкера — текст чанков включён:")
if rerank_ready is not None:
    print(rerank_ready.head(3))


ℹ️ Загружены эмбеддинги чанков: (4974, 1024)
ℹ️ FAISS index загружен


Batches:   0%|          | 0/55 [00:00<?, ?it/s]

Searching:   0%|          | 0/6977 [00:00<?, ?it/s]

✅ dense_docs сохранён -> ./artifacts/dense_docs.parquet
✅ dense_docs сохранён -> ./artifacts/dense_docs.csv
   question_id web_id  score_dense  rank_dense
0            1    372     0.607786           1
1            1   1249     0.602957           2
2            1   1567     0.599441           3
3            1     16     0.598630           4
4            1   1098     0.577506           5
5            1    777     0.576446           6
6            1    362     0.572181           7
7            1    257     0.571649           8
8            1    368     0.570179           9
9            1   1186     0.567020          10
✅ Готово: получили лучшие уникальные сайты на запрос (после дедупа по web_id)
  question_id web_id                                                url  \
0           1    372  https://alfabank.ru/help/articles/sme/rko/rass...   
1           1   1249                           https://alfabank.ru/m2m/   
2           1   1567  https://private.auth.alfabank.ru/passport/cerb... 

# RRF

In [1]:
import pandas as pd

In [2]:
base_url = "https://raw.githubusercontent.com/p0velentius/rug-pull/main/"
dense = pd.read_csv(base_url + "model/dense_results.csv")
bm25 = pd.read_csv(base_url + "model/bm25_results.csv")

In [3]:
dense.columns

Index(['question_id', 'web_id', 'score_dense', 'rank_dense'], dtype='object')

In [4]:
bm25.columns

Index(['question_id', 'web_id', 'rank_bm25', 'score_bm25'], dtype='object')

In [5]:
def rrf(
    dense: pd.DataFrame,
    bm25: pd.DataFrame,
    k: int = 60,
    alpha: float = 0.6,
    weight_dense_rrf: float = 1.0,
    weight_bm25_rrf: float = 1.0,
    weight_dense_score: float = 1.0,
    weight_bm25_score: float = 1.0,
    n: int = 20
) -> pd.DataFrame:
    """
    dense/bm25: колонки ['question_id','web_id','rank','score']
    alpha: вес RRF в итоговом скоре; (1-alpha) — вес линейного комбинирования нормализованных скоров.
    Возвращает top-n websites для всех запросов.
    """

    # merge
    merged = pd.merge(
        dense, bm25,
        on=["question_id", "web_id"],
        how="outer",
        suffixes=("_dense", "_bm25")
    )
    rank_dense  = merged["rank_dense"].astype(float)
    rank_bm25 = merged["rank_bm25"].astype(float)

    # rrf
    part_dense  = np.where(~np.isnan(rank_dense),  weight_dense_rrf  / (k + rank_dense),  0.0)
    part_bm25 = np.where(~np.isnan(rank_bm25), weight_bm25_rrf / (k + rank_bm25), 0.0)
    merged["score_rrf_part"] = part_dense + part_bm25

    # normalization
    def norm_group(g, col):
        vals = g[col].to_numpy(dtype=float)
        mask = ~np.isnan(vals)
        if not mask.any():
            return np.zeros_like(vals)

        vmin, vmax = vals[mask].min(), vals[mask].max()
        out = np.zeros_like(vals, dtype=float)

        if vmax > vmin:
            out[mask] = (vals[mask] - vmin) / (vmax - vmin)
        else:
            out[mask] = 0.0

        return out

    merged["score_dense_norm"] = merged.groupby("question_id", group_keys=False).apply(
        lambda g: pd.Series(norm_group(g, "score_dense"), index=g.index)
    )
    merged["score_bm25_norm"] = merged.groupby("question_id", group_keys=False).apply(
        lambda g: pd.Series(norm_group(g, "score_bm25"), index=g.index)
    )

    merged["score_raw_combined"] = (
        weight_dense_score  * merged["score_dense_norm"] +
        weight_bm25_score * merged["score_bm25_norm"]
    )


    # combination
    merged["score_rrf"] = alpha * merged["score_rrf_part"] + (1 - alpha) * merged["score_raw_combined"]
    merged["score_rrf"] = merged["score_rrf"].fillna(0.0)

    merged["rank_rrf"] = merged.groupby("question_id")["score_rrf"].rank(
        method="first", ascending=False
    ).astype(int)

    merged = merged.sort_values(["question_id", "score_rrf"], ascending=[True, False])
    return merged.groupby("question_id").head(n).reset_index(drop=True)


In [6]:
rrf_results = rrf(dense, bm25).loc[:, ["question_id", "web_id", "score_rrf"]].rename(
    columns={
        "question_id": "q_id",
        "score_rrf": "rrf_score"
    }
)

rrf_results.to_csv("rrf_results.csv", index=False)

/tmp/ipython-input-1645713899.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged["score_dense_norm"] = merged.groupby("question_id", group_keys=False).apply(
/tmp/ipython-input-1645713899.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged["score_bm25_norm"] = merged.groupby("question_id", group_keys=False).apply(


In [21]:
result = rrf(dense, bm25, n=5).loc[:, ["question_id", "web_id"]].rename(
    columns={"question_id": "q_id", "web_id": "web_list"}
).groupby("q_id")["web_list"].apply(list)

result.to_csv("result_after_rrf.csv")

/tmp/ipython-input-1645713899.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged["score_dense_norm"] = merged.groupby("question_id", group_keys=False).apply(
/tmp/ipython-input-1645713899.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged["score_bm25_norm"] = merged.groupby("question_id", group_keys=False).apply(
